In [1]:
from scipy import ndimage
from skimage import io,util,color
from IPython import display
import numpy
import sys
import math
import random

import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display, clear_output

import matplotlib.pyplot as plt

In [7]:
def readAndShowImage(image):
    im = io.imread(image)
    io.imshow(im)
    io.show()

inputImage = widgets.Dropdown(
    options={
        'Strand':'strand.jpeg',
        'Mandrill':'mandrill.png',
        'Trump':'trump.jpg',
        'Gruppe':'gruppe.jpg',
        'RGB':"rgb.png",
        'Dress':'TheDress.png',
        'wikipedia eksempel':'wiki.png',
        'Ulrikke':'ulrikke.jpg'
        },
    description='Bilde: ',
    disabled=False,
)

interactive(readAndShowImage, image=inputImage)

interactive(children=(Dropdown(description='Bilde: ', options={'Strand': 'strand.jpeg', 'Mandrill': 'mandrill.…

In [3]:
def gaussian(im, sigma):
    return ndimage.gaussian_filter(im,sigma)

def nonMax(M,alpha):
    gN = numpy.zeros(M.shape)
    height, width = gN.shape
        
    for i in range(height):
        for j in range(width):
            #finn retningen D som passer best med kanten i alpha
            d = alpha[i,j]
            
            if(d < 0):
                d += 360
                
            if ((j+1) < width) and ((j-1) >= 0) and ((i+1) < height) and ((i-1) >= 0):
                # 0 degrees
                if (d >= 337.5 or d < 22.5) or (d >= 157.5 and d < 202.5):
                    if M[i,j] >= M[i,j+1] and M[i,j] >= M[i,j-1]:
                        gN[i,j] = M[i,j]
                # 45 degrees
                if (d >= 22.5 and d < 67.5) or (d >= 202.5 and d < 247.5):
                    if M[i,j] >= M[i-1,j+1] and M[i,j] >= M[i+1,j-1]:
                        gN[i,j] = M[i,j]
                # 90 degrees
                if (d >= 67.5 and d < 112.5) or (d >= 247.5 and d < 292.5):
                    if M[i,j] >= M[i-1,j] and M[i,j] >= M[i+1,j]:
                        gN[i,j] = M[i,j]
                # 135 degrees
                if (d >= 112.5 and d < 157.5) or (d >= 292.5 and d < 337.5):
                    if M[i,j] >= M[i-1,j-1] and M[i,j] >= M[i+1,j+1]:
                        gN[i,j] = M[i,j]
            
                             
                             
    return gN
    

def hysterisisTresh(im, low, high):
    gNh = im >= high
    gNl = numpy.logical_and(im <= high, im >= low)
    
    retArr = numpy.zeros(im.shape)
    
    height, width = im.shape
    
    strongEdgesSet = set(())
    
    for i in range(height):
        for j in range(width):
            if(gNh[i,j]):
                strongEdgesSet.add((i,j))                
    
    while(len(strongEdgesSet) > 0):
        item = strongEdgesSet.pop()
        #adding this item to the final image since it was connected to the strong edges
        retArr[item[0],item[1]]  = True
        
        #finding weak edges connected to this pixel and adding to the set
        for i in range(-1,2):
            for j in range(-1,2):
                if(item[0] + i > 0 and item[0] + i < height and item[1] + j > 0 and item[1] + j < width):
                    if(gNl[item[0] + i, item[1] + j]):
                        #legger til i settet da det er en svak kant som er koble til en sterk
                        strongEdgesSet.add((item[0] + i, item[1] + j))
                        #Fjerner fra weak edges for å forhindre å sjekke 2 ganger
                        gNl[item[0] + i, item[1] + j] = False
    
    return retArr
    
def cannyEdge(im,sigma,low,high):
    #Finn et utjevnet gradientbilde ved  ̊a bruke førstederiverte av en Gaussian
    
    smooth = gaussian(im, sigma)
    
    sobelX, sobelY = (numpy.asarray([[-1,0,1],[-2,0,2],[-1,0,1]]),numpy.asarray([[-1,-2,-1],[0,0,0],[1,2,1]]))
    sobelX, sobelY = (ndimage.convolve(smooth,sobelX),ndimage.convolve(smooth,sobelY))
    
    M = (sobelX**2 + sobelY**2)**0.5
    alpha = numpy.arctan2(sobelY,sobelX)
    
    alpha *= 180/math.pi
    
    #Utfør nonmax suppression (ikkemax-fjerning)
    
    gN = nonMax(M,alpha)
    
    #Bruk hysteresis thresholding (dual threshold oppsett) for  ̊a finne
    canny = hysterisisTresh(gN, low, high)
    
    return canny

In [8]:
def update(image,sigma, highLow):
    im = io.imread(image,as_gray=True)
    cannyEdgeIm = cannyEdge(im,sigma, highLow[0], highLow[1])
    io.imsave('edge.png',cannyEdgeIm)
    io.imshow(cannyEdgeIm, cmap='gray')
    io.show()
    
    
highLowSlider = widgets.FloatRangeSlider(
    value=[0.1, 0.3],
    min=0,
    max=0.7,
    step=0.01,
    description='Low High Hysterisis:',
    continuous_update=False,
    orientation='horizontal'
)
    
interactive(update, image=inputImage, sigma=(1.0,2.0), highLow=highLowSlider)

interactive(children=(Dropdown(description='Bilde: ', index=7, options={'Strand': 'strand.jpeg', 'Mandrill': '…